<a href="https://colab.research.google.com/github/cakennedy/266-mbti-project/blob/main/notebook/SetFit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install setfit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Google collab premium utilities status check
# GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

print( '\n')

# RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Not connected to a GPU


Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [3]:
! ls -lrt /content/drive/MyDrive/*.json

-rw------- 1 root root 2308 Oct 13 00:53 /content/drive/MyDrive/nlpproject.json
-rw------- 1 root root 2326 Oct 31 03:37 /content/drive/MyDrive/pacific-castle-360400-a3ca89f64de6.json


In [4]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/pacific-castle-360400-a3ca89f64de6.json' 
!echo $GOOGLE_APPLICATION_CREDENTIALS

/content/drive/MyDrive/pacific-castle-360400-a3ca89f64de6.json


In [5]:
from google.cloud import storage

storage_client = storage.Client()

buckets = storage_client.list_buckets()

print('-- List of buckets in project \"' + storage_client.project + '\"')

for b in buckets:
  print(b.name)

-- List of buckets in project "pacific-castle-360400"
266csffile
w266liwc


In [6]:
#Initialize google storage
#storage_client = storage.Client.from_service_account_json('pacific-castle-360400-a3ca89f64de6.json')

#Print buckets available
for bucket in storage_client.list_buckets():
  print(bucket)

#Assign bucket name being used
bucket_name = '266csffile'
#bucket_name = 'w266liwc'

#Get bucket
bucket = storage_client.get_bucket(bucket_name)

# #Show list of files in bucket and list the files
# filelist = list(bucket.list_blobs(prefix=''))
# for name in filelist:
#   print(name.name)

<Bucket: 266csffile>
<Bucket: w266liwc>


In [7]:
import pandas as pd
import os, re
import time
import numpy as np
import pandas as pd
import io
from io import BytesIO
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score,roc_auc_score,roc_curve,recall_score, precision_score, f1_score,matthews_corrcoef
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import Dataset


In [8]:
def read_parquet_google_cloud(file):
    '''This function reads a file from the google cloud storage bucket. Input
    parameters include the filename, encoding and CSV file separators.'''

    #Load Google Cloud storage client using service key

    blob = bucket.blob(file)
    read_back = bucket.blob(file)
    string_read_back = read_back.download_as_string()
    new_df = pd.read_parquet(io.BytesIO(string_read_back))

    return new_df

In [9]:
train_mbti_data = read_parquet_google_cloud('train_uniform_mbti_data.parquet')
train_mbti_labels = read_parquet_google_cloud('train_uniform_mbti_labels.parquet')
#remove gender from the train data set
train_mbti_data = train_mbti_data[['Username','Age','Posts','Occupation','message','is_I','is_S','is_T','is_J']]
#train_mbti_data, train_mbti_labels = train_mbti_data[120000:130000], train_mbti_labels[120000:130000]
#train_mbti_data, train_mbti_labels = train_mbti_data[:40000], train_mbti_labels[:40000]
#train_mbti_data = add_features(train_mbti_data)

dev_mbti_data = read_parquet_google_cloud('dev_mbti_data.parquet')
dev_mbti_labels = read_parquet_google_cloud('dev_mbti_labels.parquet')
#dev_mbti_data, dev_mbti_labels = dev_mbti_data[:2000], dev_mbti_labels[:2000]
#remove gender from the dev data set 
dev_mbti_data = dev_mbti_data[['Username','Age','Posts','Occupation','message','is_I','is_S','is_T','is_J']]

In [10]:
train_mbti_data, train_mbti_labels = train_mbti_data[:2000], train_mbti_labels[:2000]
dev_mbti_data, dev_mbti_labels = dev_mbti_data[:500], dev_mbti_labels[:500]

In [11]:
train_mbti_data.count()

Username      2000
Age           2000
Posts         2000
Occupation    1032
message       2000
is_I          2000
is_S          2000
is_T          2000
is_J          2000
dtype: int64

In [12]:
#Create dictonary to transform MBTI type into multiclass value from 0 to 15
valid_MBTI = {'ISTJ': 0, 'INTJ': 1, 'ESTJ': 2, 'ENTJ': 3, 'ENTP': 4, 'INTP': 5, \
              'ISTP': 6, 'ESTP': 7, 'ISFJ': 8, 'INFJ': 9, 'ESFJ': 10, \
              'ENFJ': 11, 'ENFP': 12, 'INFP': 13, 'ISFP': 14, 'ESFP': 15}

In [13]:
train_mbti_data

,Username,Age,Posts,Occupation,message,is_I,is_S,is_T,is_J
0,oneandonly,37.0,109,None,i do not think i am at all. but the question...,True,False,False,True
1,thalassa,45.0,25183,tree people,YYY0 of the longest threads on this site con...,True,True,False,False
2,lady_x,46.0,18233,photographer,XXXX seems right..i would guess t/f were balan...,False,False,False,False
3,eagle,31.0,733,student,i noticed. it is hard to care over such a gr...,True,True,True,True
4,ayoitsstepho,31.0,4838,painter,:hug:,True,True,False,False
...,...,...,...,...,...,...,...,...,...
1995,yuurei,39.0,4509,artist,"dear deer, i have not seen much of you this y...",False,False,True,True
1996,ejcc,31.0,19129,None,i do kind of see sx/sp being especially vulner...,False,True,True,True
1997,tilt,32.0,2585,None,"well you got me beat, i failed out of high s...",False,False,False,True
1998,baj,51.0,626,"aquaculturist, biologist, gardener, landscaper...","the duet it is true, the rockies are rather l...",True,True,False,False


In [14]:
train_combined = pd.concat([train_mbti_data,train_mbti_labels],axis=1)

In [15]:
train_combined['labels']= train_combined['MBTI Type'].map(valid_MBTI)

In [16]:
train_combined = pd.concat([train_mbti_data,pd.get_dummies(train_combined['MBTI Type'])],axis=1)

In [17]:
train_combined.count()

Username      2000
Age           2000
Posts         2000
Occupation    1032
message       2000
is_I          2000
is_S          2000
is_T          2000
is_J          2000
ENFJ          2000
ENFP          2000
ENTJ          2000
ENTP          2000
ESFJ          2000
ESFP          2000
ESTJ          2000
ESTP          2000
INFJ          2000
INFP          2000
INTJ          2000
INTP          2000
ISFJ          2000
ISFP          2000
ISTJ          2000
ISTP          2000
dtype: int64

In [18]:
train_features = train_combined[['message','ENFJ','ENFP','ENTJ','ENTP','ESFJ','ESFP','ESTJ','ESTP','INFJ','INFP','INTJ','INTP','ISFJ','ISFP','ISTJ','ISTP']]

In [19]:
dev_combined = pd.concat([dev_mbti_data,dev_mbti_labels],axis=1)

In [20]:
dev_combined['labels']= dev_combined['MBTI Type'].map(valid_MBTI)

In [21]:
dev_combined = pd.concat([dev_mbti_data,pd.get_dummies(dev_combined['MBTI Type'])],axis=1)

In [22]:
dev_combined

,Username,Age,Posts,Occupation,message,is_I,is_S,is_T,is_J,ENFJ,...,ESTJ,ESTP,INFJ,INFP,INTJ,INTP,ISFJ,ISFP,ISTJ,ISTP
1200000,ajblaise,35.0,7914,web development,thank you for this perspective. it is aston...,True,False,True,False,0,...,0,0,0,0,0,1,0,0,0,0
1200001,red memories,24.0,6315,record store guru,[mediayoutube]YYYezewYYYkopui[/media],False,True,False,False,0,...,0,0,0,0,0,0,0,0,0,0
1200002,thatswhathesaid,42.0,7256,graphic design,"oh, i would also like to add that they are the...",True,False,True,False,0,...,0,0,0,0,0,1,0,0,0,0
1200003,lowtech redneck,44.0,3711,None,revysleetles is trying to register a new accou...,True,False,True,False,0,...,0,0,0,0,0,1,0,0,0,0
1200004,synarch,45.0,8447,None,"repost and this is basic, well know stuff here...",False,False,True,False,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200495,aleksei,33.0,3626,poker player,"well, it says you are in my pants, and my pant...",False,False,True,True,0,...,0,0,0,0,0,0,0,0,0,0
1200496,phoenix13,37.0,1293,doc,woah woah woah! XXXX.. community arts teac...,False,False,False,False,0,...,0,0,0,0,0,0,0,0,0,0
1200497,cafe,52.0,9829,social justice warrior,i finally got the bright idea to look up the...,True,False,False,True,0,...,0,0,1,0,0,0,0,0,0,0
1200498,skylights,34.0,7759,healthcare & student,":alttongue: i am watching local tv, keeping...",True,False,False,False,0,...,0,0,0,1,0,0,0,0,0,0


In [23]:
dev_features = dev_combined[['message','ENFJ','ENFP','ENTJ','ENTP','ESFJ','ESFP','ESTJ','ESTP','INFJ','INFP','INTJ','INTP','ISFJ','ISFP','ISTJ','ISTP']]

In [24]:
t_dataset = Dataset.from_pandas(train_features)

In [25]:
features = t_dataset.column_names
features.remove("message")
features

['ENFJ',
 'ENFP',
 'ENTJ',
 'ENTP',
 'ESFJ',
 'ESFP',
 'ESTJ',
 'ESTP',
 'INFJ',
 'INFP',
 'INTJ',
 'INTP',
 'ISFJ',
 'ISFP',
 'ISTJ',
 'ISTP']

In [26]:
def encode_labels(record):
    return {"labels": [record[feature] for feature in features]}


t_dataset = t_dataset.map(encode_labels)

  0%|          | 0/2000 [00:00<?, ?ex/s]

In [27]:
t_dataset

Dataset({
    features: ['message', 'ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP', 'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP', 'labels'],
    num_rows: 2000
})

In [28]:
v_dataset = Dataset.from_pandas(dev_features)

In [29]:
v_dataset = v_dataset.map(encode_labels)

  0%|          | 0/500 [00:00<?, ?ex/s]

In [30]:
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers.losses import CosineSimilarityLoss
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2",multi_target_strategy="one-vs-rest")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [31]:

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=t_dataset,
    eval_dataset=v_dataset,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=8,
    num_iterations=2, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for constrastive learning
    column_mapping={"message": "text", "labels": "label"},
)



In [ ]:
# Train and evaluate
trainer.train()




Applying column mapping to training dataset
***** Running training *****
  Num examples = 8000
  Num epochs = 1
  Total optimization steps = 1000
  Total train batch size = 8


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1000 [00:00<?, ?it/s]

In [1]:
metrics = trainer.evaluate()

NameError: ignored

In [2]:

print(f"train dataset: {len(t_dataset)} samples")
print(f"accuracy: {metrics['accuracy']}")

NameError: ignored

# Sample Code 

In [39]:
from datasets import load_dataset
dataset = load_dataset("ethos", "multilabel")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/433 [00:00<?, ? examples/s]

Dataset ethos downloaded and prepared to /root/.cache/huggingface/datasets/ethos/multilabel/1.0.0/898d3d005459ee3ff80dbeec2f169c6b7ea13de31a08458193e27dec3dd9ae38. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
dataset['train'][0]

{'text': "You should know women's sports are a joke",
 'violence': 0,
 'directed_vs_generalized': 0,
 'gender': 1,
 'race': 0,
 'national_origin': 0,
 'disability': 0,
 'religion': 0,
 'sexual_orientation': 0}

In [54]:
features = dataset["train"].column_names
features.remove("text")
features

['violence',
 'directed_vs_generalized',
 'gender',
 'race',
 'national_origin',
 'disability',
 'religion',
 'sexual_orientation']

In [55]:
num_samples = 8
samples = np.concatenate(
    [np.random.choice(np.where(dataset["train"][f])[0], num_samples) for f in features]
)

In [56]:
samples

array([276, 144,  57, 346, 168, 399, 215,  81,  31, 334, 415, 269, 389,
       394,  94, 319, 101, 336, 389, 295, 117, 388,  64, 126, 221,  93,
       321, 121, 161, 121, 199,  82, 331, 343, 343, 114,  17, 289, 278,
       252, 159, 280, 326, 258,  98, 279, 422, 159, 158, 266, 228, 303,
       308, 303, 296, 430, 347, 393,  77,  25, 176, 106, 196, 427])

In [57]:
def encode_labels(record):
    return {"labels": [record[feature] for feature in features]}


dataset = dataset.map(encode_labels)

  0%|          | 0/433 [00:00<?, ?ex/s]

In [60]:
dataset['train'][0]

{'text': "You should know women's sports are a joke",
 'violence': 0,
 'directed_vs_generalized': 0,
 'gender': 1,
 'race': 0,
 'national_origin': 0,
 'disability': 0,
 'religion': 0,
 'sexual_orientation': 0,
 'labels': [0, 0, 1, 0, 0, 0, 0, 0]}

In [62]:
train_features = train_combined[['message','ENFJ','ENFP','ENTJ','ENTP','ESFJ','ESFP','ESTJ','ESTP','INFJ','INFP','INTJ','INTP','ISFJ','ISFP','ISTJ','ISTP']]

In [73]:
dev_features = dev_combined[['message','ENFJ','ENFP','ENTJ','ENTP','ESFJ','ESFP','ESTJ','ESTP','INFJ','INFP','INTJ','INTP','ISFJ','ISFP','ISTJ','ISTP']]

KeyError: ignored

In [63]:
train_features 

,message,ENFJ,ENFP,ENTJ,ENTP,ESFJ,ESFP,ESTJ,ESTP,INFJ,INFP,INTJ,INTP,ISFJ,ISFP,ISTJ,ISTP
0,i do not think i am at all. but the question...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,YYY0 of the longest threads on this site con...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,XXXX seems right..i would guess t/f were balan...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,i noticed. it is hard to care over such a gr...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,:hug:,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,am i really bipolar? is this just a low? it se...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
96,"[mediayoutube]YYYobbeopsuw, list: rdYYYlssdvYY...",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
97,just because you do not understand fe or lik...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
98,"yeah, not okay. should we just dumpster the wh...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
